In [25]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

from numba import njit, jit
from numpy.linalg import norm

from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import lr_scheduler
from torchmetrics import MeanAbsolutePercentageError

import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

from utility_funcs import *
from LJ_modeling_realization.includes.constants import *

import dill

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ConstantKernel, RBF
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Normalizer

In [26]:
N = CFG.N

In [27]:
# dt = float(input("integrated dt:"))

In [28]:
model_type = "gpr"

In [29]:
class SklearnModel:
    def __init__(self, model):
        '''
        model - sklearn model
        '''
        self.model = model

    def __call__(self, X):
        return self.model.predict(X)

In [30]:
model_file_path = f'./trained_models/{model_type}_2_movements_K3.pickle'
model_vel_file_path = f'./trained_models/{model_type}_2_velocities_K3.pickle'

# Сетка предсказания сжатые будет выдавать и их надо будет возвращать к обычному скейлу
descaler_path = f'./trained_models/descaler_{CFG.N}_K{CFG.K}.pickle'
descaler_vel_path = f'./trained_models/descaler_vel_{CFG.N}_K{CFG.K}.pickle'

In [31]:
with open(model_file_path, 'rb') as handle:
    model = dill.load(handle)

with open(model_vel_file_path, 'rb') as handle:
    model_vel = dill.load(handle)

if model_type == "gpr":
    descaler = Descaler(1, 0)
    descaler_vel = Descaler(1, 0)
    model = SklearnModel(model)
    model_vel = SklearnModel(model_vel)
else:
    with open(descaler_path, 'rb') as handle:
        descaler = dill.load(handle)
    

In [32]:
def csv_row_to_state(coords_path, vels_path, row_number=0):
    '''
    This function will be mostly used to start integration and create header for csv to write into
    '''

    row_coords = np.array(pd.read_csv(
        coords_path
    ).iloc[row_number, :][1:]).reshape(CFG.N, 3)

    row_vels = np.array(pd.read_csv(
        vels_path
    ).iloc[0, :][1:]).reshape(CFG.N, 3)
    
    state = {i: [row_coords[i], row_vels[i]] for i in range(CFG.N)}


    return state

In [33]:
n_state = csv_row_to_state(
    f'./coords_and_movements/coords{CFG.N}.csv',
    f'./coords_and_movements/velocities{CFG.N}.csv',

    row_number=-1   # НАЧИНАЕМ ПРЕДСКАЗЫВАТЬ С КОНЦА ИЗВЕСТНОЙ ТРАЕКТОРИИ
)

In [34]:
def get_distances(n_state, atom_numb):
    '''
    Возвращает все расстояния от данного атома
    '''
    pass

In [35]:
# @njit
def get_closest_atom(state, atom_number):
    other_atom_numbers = [i for i in range(CFG.N) if i != atom_number]
    Rel_matrix = np.zeros([CFG.N - 1, 3])

    for i, other_numb in enumerate(other_atom_numbers):
        Rel_matrix[i] = state[atom_number][0] - state[other_numb][0]

    # Getting the closest atom to atom_number:
    dists = norm(Rel_matrix, axis=1)

    return np.argmin(dists)

In [36]:
def _get_relative_positions(state, atom_number):
    '''
    This function processes one row of csv into something that we can work with

    Returns np.array matrix that consists of relative positions vectors for passed atom_number to every other atom
    and then we can chose only closest N_neighbours in the next functions
    
    row: df.iloc[row] - typeof(row): pd.Series
    
    returns: Rel_matrix, f_vec
    '''

    other_atom_numbers = [i for i in range(CFG.N) if i != atom_number]
    Rel_matrix = np.zeros([CFG.N - 1, 3])

    for i, other_numb in enumerate(other_atom_numbers):
        Rel_matrix[i] = state[atom_number][0] - state[other_numb][0]

    return np.array(Rel_matrix)

def create_V_i(i, normalized_m, norms, r_cut=CFG.r_cut, p=CFG.p):
    '''
    normalized_m: matrix of relative distances, where rows - normalized vectors
    i: i-th component of r_cut and p, i in range 1..K (or in 0..K-1 in code)
    '''
    transf_vecs = make_matrix_transformed(normalized_m, norms[:, np.newaxis], r_cut[i], p[i])

    return np.sum(transf_vecs, axis=0)

# @njit(parallel=True)
def create_V(normalized_m, norms, K=CFG.K):
    '''
    creates V
    '''
    V = []
    for i in range(K):
        V.append(
            create_V_i(i, normalized_m, norms)
        )

    return np.stack(V)

In [37]:
def make_step(n_state):
    pass

In [38]:
n_state[0]

[array([1.9533191 , 0.72123262, 0.26282041]),
 array([ 0.51596858, -0.16688677,  0.01485316])]

**Короче: перемещения надо брать из unfolded координат, матрицы признаков надо брать из folded координат, а мб я и не прав**

In [51]:
# ЗАПУСКАЮ ЦИКЛ, СЧИТАЮ ВСЕ РАССТОЯНИЯ, ЕСЛИ ОНО МИНИМАЛЬНОЕ - СТРОЮ state для текущий частицы с этой частицей - делаю предсказание для этой частицы, но пока не сдвигаю, так делаю для всех частиц и затем сдвигаю

def n_integration_from_2(n_state, steps):
    for atom_num in range(N):
        state_2p = {}
        other_atom_for_2p_state = get_closest_atom(n_state, atom_num)
        state_2p = {0: n_state[atom_num], 1: n_state[other_atom_for_2p_state]}

        print(state_2p)
    return state_2p

In [52]:
a = n_integration_from_2(n_state, 1)

{0: [array([1.9533191 , 0.72123262, 0.26282041]), array([ 0.51596858, -0.16688677,  0.01485316])], 1: [array([ 3.748383  , 10.76555622,  2.30140656]), array([1.66959643, 3.94339942, 0.8335923 ])]}
{0: [array([ 0.78181899, -2.67334847, -2.47903861]), array([-0.8541372 , -0.44242995, -0.57602303])], 1: [array([1.9533191 , 0.72123262, 0.26282041]), array([ 0.51596858, -0.16688677,  0.01485316])]}
{0: [array([ 5.1106208 , -2.79698332,  1.93020422]), array([ 0.77939097, -2.27582507,  0.06638178])], 1: [array([-4.01083052,  3.0175924 ,  1.12036337]), array([-2.36451652,  1.38732474, -1.31930425])]}
{0: [array([8.84820201, 0.63445318, 2.44737825]), array([1.20623632, 0.16722855, 0.25292309])], 1: [array([-0.46768814,  5.02130211,  2.88672862]), array([-2.79751058,  1.25891468,  0.34888104])]}
{0: [array([-1.29286337,  3.31618813,  2.44679096]), array([-1.25183726,  0.07842462,  0.36682924])], 1: [array([6.17393534, 5.17558735, 6.29582929]), array([ 0.85888468, -0.10291455,  1.10414305])]}
{0: